<a href="https://colab.research.google.com/github/padolekrushna/Prompt-Engineering-Class/blob/main/HOMEWORK_3_Word_Embeddings_and_RAG_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link to google colab : https://colab.research.google.com/drive/14sWo11mB8gFwq3elZp9Cbw3CvghiSdly?usp=sharing

#Part 1: Word Embedding Arithmetic

##Step 1: Load the BERT Model and Tokenizer

In [1]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Ensure the model is in evaluation mode
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

#Step 2: Implement Functions to Get Word Embeddings and Perform Word Arithmetic

In [2]:
# Function to get the embedding of a word
def get_word_embedding(word):
    # Tokenize the word and convert to tensor
    inputs = tokenizer(word, return_tensors='pt')

    # Get the hidden states from BERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the embeddings from the last hidden layer, taking the average of the token embeddings
    word_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()

    return word_embedding

# Function to perform word arithmetic
def word_arithmetic(words1):
    vec = get_word_embedding(words1[0])

    for i in range(1, len(words1), 2):
        if words1[i] == '-':
            vec -= get_word_embedding(words1[i + 1])
        elif words1[i] == '+':
            vec += get_word_embedding(words1[i + 1])

    return vec

# Function to find the most similar word
def find_most_similar(target_embedding, words_list):
    similarities = []

    for word in words_list:
        word_embedding = get_word_embedding(word)
        # Compute cosine similarity
        cos_sim = torch.nn.functional.cosine_similarity(target_embedding, word_embedding, dim=0)
        similarities.append((word, cos_sim.item()))

    # Sort based on similarity and return the most similar word
    similarities.sort(key=lambda x: x[1], reverse=True)

    return similarities[0][0]


#Step 3: Perform Word Arithmetic and Find Most Similar Word

In [3]:
# List of examples
examples = [
    (["paris", "-", "france", "+", "italy"], ["rome", "romaine", "romania", "ronnie", "random"]),
    (["king", "-", "man", "+", "woman"], ["queen", "lady", "empress", "princess", "duchess"]),
    (["apple", "-", "fruit", "+", "vegetable"], ["carrot", "potato", "broccoli", "grape", "berry"]),
    (["japan", "-", "tokyo", "+", "seoul"], ["korea", "china", "singapore", "taiwan", "india"]),
    (["doctor", "-", "hospital", "+", "school"], ["teacher", "student", "principal", "nurse", "librarian"])
]

# Execute the word arithmetic for each example
for i, (arithmetic_words, candidates) in enumerate(examples):
    target_embedding = word_arithmetic(arithmetic_words)
    most_similar_word = find_most_similar(target_embedding, candidates)
    print(f"Example {i+1}: {arithmetic_words} -> Most similar word: {most_similar_word}")


Example 1: ['paris', '-', 'france', '+', 'italy'] -> Most similar word: rome
Example 2: ['king', '-', 'man', '+', 'woman'] -> Most similar word: queen
Example 3: ['apple', '-', 'fruit', '+', 'vegetable'] -> Most similar word: potato
Example 4: ['japan', '-', 'tokyo', '+', 'seoul'] -> Most similar word: korea
Example 5: ['doctor', '-', 'hospital', '+', 'school'] -> Most similar word: teacher


#Part 2: RAG System Implementation